## Calculate the PCA's for the MWE regenie statgen-courses

In [6]:
module load Singularity

In [8]:
sos run ~/project/xqtl-pipeline/code/data_preprocessing/genotype/GWAS_QC.ipynb king \
    --cwd ~/mwe_regenie_statgen/output \
    --genoFile ~/mwe_regenie_statgen/1000G.EUR.mwe.pruned.bed\
    --kinship 0.13 \
    --name regenie_statgen \
    --container ~/containers/bioinfo.sif

INFO: Running king_1: Inference of relationships in the sample to identify closely related individuals
INFO: king_1 is completed.
INFO: king_1 output:   /home/dmc2245/mwe_regenie_statgen/output/1000G.EUR.mwe.pruned.regenie_statgen.kin0
INFO: Running king_2: Select a list of unrelated individual with an attempt to maximize the unrelated individuals selected from the data
ERROR: king_2 (id=a6aafb8b3037f0cf) returns an error.
ERROR: [king_2]: [0]: Executing script in Singularity returns an error (exitcode=1, stderr=/home/dmc2245/mwe_regenie_statgen/output/1000G.EUR.mwe.pruned.regenie_statgen.related_id.stderr).
The script has been saved to /home/dmc2245/.sos/fcec955dd263e8d4/singularity_run_14181.R. To reproduce the error please run:
singularity exec  /home/dmc2245/containers/bioinfo.sif Rscript /home/dmc2245/.sos/fcec955dd263e8d4/singularity_run_14181.R
[king]: Exits with 1 pending step (king_3)


: 1

In [ ]:
sos run GWAS_QC.ipynb qc \
    --cwd ~/mwe_regenie_statgen/output \
    --genoFile ~/mwe_regenie_statgen/rename_chr22.20220110.unrelated.bed \
    --maf-filter 0.01 \
    --name for_pca \
    --container container/bioinfo.sif

In [ ]:
sos run GWAS_QC.ipynb qc_no_prune \
    --cwd ~/mwe_regenie_statgen/output \
    --genoFile ~/mwe_regenie_statgen/output/rename_chr22.20220110.related.bed \
    --keep-variants output/rename_chr22.20220110.unrelated.for_pca.filtered.prune.in \
    --maf-filter 0 --geno-filter 0 --mind-filter 0.1 --hwe-filter 0 \
    --name for_pca \
    --container container/bioinfo.sif

In [12]:
sos run ~/project/xqtl-pipeline/code/data_preprocessing/genotype/PCA.ipynb flashpca \
        --cwd ~/mwe_regenie_statgen/output/pca \
        --genoFile ~/mwe_regenie_statgen/1000G.EUR.mwe.pruned.bed \
        --phenoFile ~/mwe_regenie_statgen/1000G.EUR.pheno.x \
        --maha-k 2 \
        --container ~/containers/flashpcaR.sif

INFO: Running flashpca_1: Run PCA analysis using flashpca
INFO: flashpca_1 is completed.
INFO: flashpca_1 output:   /home/dmc2245/mwe_regenie_statgen/output/pca/1000G.EUR.pheno.pca.rds
INFO: Running flashpca_2: 
INFO: flashpca_2 is completed (pending nested workflow).
INFO: Running detect_outliers: Calculate Mahalanobis distance per population and report outliers
INFO: detect_outliers is completed.
INFO: detect_outliers output:   /home/dmc2245/mwe_regenie_statgen/output/pca/1000G.EUR.pheno.pca.mahalanobis.rds /home/dmc2245/mwe_regenie_statgen/output/pca/1000G.EUR.pheno.pca.outliers... (5 items)
INFO: flashpca_2 output:   /home/dmc2245/mwe_regenie_statgen/output/pca/1000G.EUR.pheno.pca.mahalanobis.rds /home/dmc2245/mwe_regenie_statgen/output/pca/1000G.EUR.pheno.pca.outliers... (5 items)
INFO: Running flashpca_3: 
INFO: flashpca_3 is completed (pending nested workflow).
INFO: Running plot_pca: Plot PCA results. Can be used independently as "plot_pca" or combined with other workflow as eg

In [ ]:
sos run PCA.ipynb project_samples \
  --cwd output/pca \
  --genoFile output/rename_chr22.20220110.related.for_pca.filtered.extracted.bed \
  --phenoFile data/MWE_pheno.txt \
  --pca-model output/pca/MWE_pheno.pca.rds \
  --label-col RACE \
  --pop-col RACE \
  --maha-k 2 \
  --container container/flashpcaR.sif